In [1]:
%matplotlib ipympl
def figure(name, nrows=1, ncols=1, *args, **kwargs):
    plt.close(name)
    return plt.subplots(nrows, ncols, num=name, *args, **kwargs)

import numpy as np
import pylab as plt
plt.style.use('default')

In [2]:
from mpl_toolkits.mplot3d import Axes3D
from functools import reduce
from numba import njit
from ipywidgets import HBox, IntSlider, FloatSlider, FloatLogSlider, RadioButtons, Select

Der folgende Code besteht im Wesentlichen aus ein paar mehr oder weniger erfolglosen Versuchen, zwei sich schneidene Flächen mit Hilfe von `matplotlib` darzustellen. Um das Video am Ende des notebooks einzufügen, müssen Sie zuerst `Ellipsoid_pv.ipynb` ausführen.

In [3]:
def make_ellipsiod(a, b, c, N, full=True):
    if full:
        theta = np.linspace(0, np.pi, N)
    else:
        theta = np.linspace(0, np.pi/2, N)
    phi = np.linspace(0, 2*np.pi, N)
    #r = np.linspace(0, 1, N)
    The, Phi= np.meshgrid(theta, phi)
    X = a * np.sin(The) * np.cos(Phi)
    Y = b * np.sin(The) * np.sin(Phi)
    Z = c * np.cos(The)
    return np.array([X, Y, Z])

In [4]:
I1, I2, I3 = 1, 1.5, 2
E = 1
M = np.sqrt(2*E*1.1)
paras = [np.sqrt(2*E*I1), np.sqrt(2*E*I2), np.sqrt(2*E*I3)]

L = make_ellipsiod(*paras, 40)
K = make_ellipsiod(M, M, M, 40)

plt.close('test_rot')
fig = plt.figure('test_rot', figsize=(5, 5))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(*L, alpha=0.5)
ax.plot_surface(*K, alpha=0.5)
# Achsen skalieren
ax.set_box_aspect([np.ptp(Li) for Li in L]) 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
Ll = L.reshape((3, 40*40))
Kl = K.reshape((3, 40*40))

plt.close('test_scat')
fig = plt.figure('test_scat', figsize=(5, 5))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(*Ll, alpha=0.5)
ax.scatter(*Kl, alpha=0.5)

# Achsen skalieren
ax.set_box_aspect([np.ptp(Li) for Li in L]) 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
@njit(cache=True)
def search_close(E, R, eps=1e-1):
    E = E.reshape((3, E.shape[1]**2))
    points = []
    for i in range(E.shape[1]):
        z = np.sqrt(R**2 - E[0, i]**2 - E[1, i]**2)
        if np.abs(E[2, i] - z) < eps:
            points.append(E[:, i].copy())
            points.append(E[:, i].copy())
            points[-1][2] = - points[-1][2] 
    return points
points = np.array(search_close(L, M))

In [7]:
points.shape

(56, 3)

In [8]:
plt.close('test_scatter')
fig = plt.figure('test_scatter', figsize=(5, 5))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(*L, alpha=0.1)
# Achsen skalieren
ax.set_box_aspect([np.ptp(Li) for Li in L]) 
N = 1000
L2 = make_ellipsiod(*paras, N)
Ms = np.linspace(np.sqrt(2 * E * I1), np.sqrt(2 * E * I3), 7)
for M in Ms:
    ax.scatter(*np.array(search_close(L2, M, eps=0.01)).T)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
def get_slider(name, L):
    M_s = FloatSlider(
        orientation='horizontal', description='$M$: ',
        value=np.sqrt(2*E*I1), min=np.sqrt(2*E*I1), max=np.sqrt(2*E*I3), step = 0.005
    )

    plt.close(name)
    fig = plt.figure(name, figsize=(6, 7))
    ax = fig.add_subplot(111, projection='3d')
    
    ax.plot_surface(*L, alpha=0.2)
    N = 500
    L2 = make_ellipsiod(*paras, N)
    sc = ax.scatter(*np.array(search_close(L2, M_s.value, eps=0.01)).T, color='black')
    ax.set_box_aspect([np.ptp(Li) for Li in L]) 

    def redraw(change):
        points = np.array(search_close(L2, M_s.value, eps=0.01)).T
            
        sc._offsets3d = points
            
        fig.canvas.draw()
        fig.canvas.flush_events()
        
    M_s.observe(redraw, names='value')
    return M_s
get_slider('cuts1', L)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

FloatSlider(value=1.4142135623730951, description='$M$: ', max=2.0, min=1.4142135623730951, step=0.005)

In [10]:
I1, I2, I3 = 1, 1, 2
E = 1
paras = [np.sqrt(2*E*I1), np.sqrt(2*E*I2), np.sqrt(2*E*I3)]
L = make_ellipsiod(*paras, 40)
get_slider('cuts2', L)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

FloatSlider(value=1.4142135623730951, description='$M$: ', max=2.0, min=1.4142135623730951, step=0.005)

<video controls src="pics/out.mp4" />